In [3]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

<IPython.core.display.Javascript object>

In [4]:
import os,sys
sys.path.append('./misc/lib/python3.7/site-packages')

%matplotlib inline

import math
import numpy as np
import requests
import ipywidgets as widgets
import time
import parmed as pmd
import re

import matplotlib.pyplot as plt
from IPython.display import display, display_markdown
from ipywidgets import Layout, HTML
from pathlib import Path
from scipy import spatial

import threading

NGL_DEF = False
try:
    import nglview as nv
    NGL_DEF = True
except:
    NGL_DEF = False

import hublib.use
%use gromacs-2018.4

np.set_printoptions(precision=8)
np.set_printoptions(suppress=True)


HTMLButtonPrompt = '''<html>
<head>
<meta name="viewport" content="width=device-width, initial-scale=1">
</head>
<body>
<a href="{link}" target="_blank" >
<button class="p-Widget jupyter-widgets jupyter-button widget-button mod-warning" style="width:150px; background-color:#CCCCCC; font-size:10pt; color:black">{text}</button>
</a>
</body>
</html>
'''   


In [5]:
def writelog(msg, leadchar):
    with open("./logfile.txt", 'a') as fd:
        for line in msg:
            fd.write(leadchar + " " + line + "\n")
            
def runbash(cmd, comment):
    out = !{cmd}
    writelog(["*****************************************************"], '')
    writelog([comment], '# ')
    writelog(["Running bash command:"], '# ')
    writelog([cmd+'\n'], 'bash >> ')
    writelog(out, 'out: ')
    writelog(["*****************************************************"], '')
    return out

In [ ]:
# add_hydrogens parses the molecular structure and adds hydrogens based on an assumed hybridization state 
# determined from the number of bonds to the parent C atom in the itp file. Any hydrogens in the original
# input structure are deleted. 

def add_hydrogens():
    
    pfile = 'pigments.pdb'
    
    # return message
    mess = ''
    
    # error flag
    error = False
    
    struc = pmd.load_file(pfile)

    # H atoms will be added 1.1 Angstroms from C
    hbond_length = 1.1

    # Create an empty structure for the pigment data
    pigstruc = pmd.structure.Structure()

    for res in struc.residues:
        tfile = res.name + '.itp'
        
        # Check whether *.itp file exists. If not, throw an error
        if os.path.exists(tfile)==False:
            mess += 'Could not locate itp file for pigment ' + res.name + '\n'
            error = True
            break
        
        # If *.itp is located, load atom and bond data from it
        else:
            
            prfile = 'posre_' + res.name + '.itp'
            # Check whether posre_*.itp file exists. If not, note that one needs to be created
            if os.path.exists(prfile)==False:
                MAKE_POSRE = True
            else:
                MAKE_POSRE = False


            # First load atom data
            with open(tfile) as tfd:
                in_atoms = False
                
                # AtNames and AtNums store atom names and numbers from *.itp file
                AtNums = []
                AtNames = []
                for line in tfd:
                    if in_atoms and line[0]=='[':
                        in_atoms = False

                    if in_atoms and len(line.split())>0 and line.split()[0].isdigit():
                        AtNums.append(int(line.split()[0]))
                        AtNames.append(line.split()[1])

                    if line[0:9]=='[ atoms ]':
                        in_atoms = True
            tfd.close()        
            AtNums = np.array(AtNums)

            # Now load bond data
            with open(tfile) as tfd:
                in_bonds = False
                
                # Bonds stores bond data from *.itp file
                Bonds = []
                for line in tfd:
                    if in_bonds and line[0]=='[':
                        in_bonds = False
                    
                    if in_bonds and len(line.split())>0 and line.split()[0].isdigit():
                        dat = [int(line.split()[0]), int(line.split()[1])]
                        Bonds.append(dat)

                    if line[0:9]=='[ bonds ]':
                        in_bonds = True
            tfd.close()
            Bonds = np.array(Bonds)
            
            # We construct a new pigment struc that contains properly named/ordered hydrogens
            # **Any hydrogens in the original structure are ignored.**
            if error==False:
                
                # Count how many hydrogens have been added to this pigment struc
                hadded = 0
                
                # Generate an empty structure for this pigment
                newstruc = pmd.structure.Structure()

                # Generate an empty coordinates list for atom positions
                coords = []

                # This will count how far we are through the atoms of the original residue
                # Note that we rely on the assumption that heavy atoms are in the same order
                # in itp file and pdb file!
                pdb_ndx = 0
                
                # HeavyParts and HydroParts will store a list of bonding partners
                # for each atom in the itp file. 
                HeavyParts = []  # Index in parmed struc of heavy-atom partners
                HydroParts = []  # Index in parmed struc of hydrogen partners
                
                # Step through atoms listed in itp file
                for n in range(0, len(AtNames)):
                    name = AtNames[n]
                    num = AtNums[n]
                    
                    # If it's a heavy atom, find the corresponding atom in the input struc and 
                    # copy it over to the new newstruc. If it's a H, we just ignore it. 
                    if name[0]!='H':
                        newat = res.atoms[pdb_ndx]
                        newstruc.add_atom(newat, res.name, res.number, chain=res.chain)
                        coords.append(struc.coordinates[struc.atoms.index(newat)])
                        pdb_ndx += 1
                        
                        # blist contains itp atom numbers of bonding partners
                        blist = Bonds[np.where((Bonds[:,0]==num)+(Bonds[:,1]==num))]
                        
                        # Now we generate a list of both heavy-atom and hydrogen-atom bond partners
                        heavy_partners = []
                        hydro_partners = []
                        for bond in blist:
                            # If bond[0]==num, then the partner is bond[1]
                            if bond[0]==num:
                                part = bond[1]
                                
                            # Otherwise, the partner is bond[0]
                            else:
                                part = bond[0]
                            
                            # If this is a hydrogen atom, add it to hydro_partners
                            if AtNames[part-1][0]=='H':
                                hydro_partners.append(part)
                            
                            # Otherwise, add it to heavy_partners
                            else:
                                heavy_partners.append(part)

                        # Update bonding partners in partner lists
                        # Note that we subtract one to make these array indices, since
                        # itp file indices start at 1 instead of 0. 
                        HeavyParts.append(np.array(heavy_partners)-1)
                        HydroParts.append(np.array(hydro_partners)-1)

                        # If there are hydrogen partners to be added, add them now to newstruc. 
                        if len(hydro_partners)>0:
                            
                            # The current atom is the "parent" for the new H atom
                            parent_index = len(HeavyParts)-1
                            
                            # For each H atom bonded to the parent, add a new atom to newstruc
                            for h in range(0, len(hydro_partners)):
                                
                                # Added H atoms are named H1, H2, H3,...
                                newat = pmd.topologyobjects.Atom(name='H'+str(hadded+1), atomic_number=1)
                                
                                # Copy all residue data (name, number, chain) from the parent residue
                                newstruc.add_atom(newat, res.name, res.number, chain=res.chain)
                                
                                # Add a blank set of coordinates for the new atom. We'll set these later.
                                coords.append([0.0, 0.0, 0.0])
                                
                                # Each H has exactly one heavy partner, i.e., its parent
                                HeavyParts.append([parent_index])
                                
                                # Each H has no hydrogen-bonding partners
                                HydroParts.append([])
                                
                                # Increment hadded
                                hadded += 1
                           
                # Once we've finished adding atoms, update the coordinate array in newstruc
                newstruc.coordinates = np.array(coords)

                # Now all atoms are added, but H atom coordinates are not set. 
                # We need now to calculate H positions from bonding partners
                
                # Make a copy of the original coordinate array
                coords = newstruc.coordinates.copy()
                
                # Step through atoms in the itp list. These should now match entries in newstruc
                for n in range(0, len(AtNames)):
                    
                    # at is the Parmed atom object for the atom in question
                    at = newstruc.atoms[n]
                    
                    # If this is a heavy atom, we'll now determine coordinates for all its Hs.
                    # By construction, these will be the next Nhydros atoms in the structure. 
                    # If it's a hydrogen, just move on -- we'll have already set its coordinates
                    # by the time we reach it. 
                    if at.name!='H':
                        
                        # How many hydrogens does it bond to? We need to set this many coordinates. 
                        Nhydros = len(HydroParts[n])
                        
                        # How many heavy atoms does it bond to? We need this to determine hybrydization geometry.
                        Nheavys = len(HeavyParts[n])

                        # parent atom coordinates
                        xpar = newstruc.coordinates[n]

                        # First calculate average position of heavy-atom bond partners
                        com = np.zeros((3,))
                        for part in HeavyParts[n]:
                            com += newstruc.coordinates[part]
                        com /= Nheavys
                        
                        # part_axis points from the heavy-partner center of mass to the parent. 
                        part_axis = (xpar - com)/np.linalg.norm(xpar - com)

                        # If only one hydrogen, desired bond axis points from 
                        # average position of bonding partners to the parent atom,
                        # i.e., along part_axis. 
                        if Nhydros==1:
                            coords[n+1] = xpar + part_axis*hbond_length

                        # If two hydrogens, we have to determine the hybridization state
                        if Nhydros==2:

                            # SP2 hybridization
                            if (Nhydros+Nheavys)==3:

                                # To define bonding plane, we need a 
                                # heavy-atom bond partner FOR the bond partner

                                # part1 is the first heavy-atom partner of the parent
                                part1 = HeavyParts[n][0]
                                xp1 = newstruc.coordinates[part1]

                                # part2 is the first heavy-atom partner of part1
                                part2 = HeavyParts[part1][0]
                                xp2 = newstruc.coordinates[part2]

                                # xpar, xp1, and xp2 define the bonding plane
                                nvec = np.cross(xpar-xp1, xp2-xp1)
                                nvec /= np.linalg.norm(nvec)

                                # Rotation objects for transforming H-atom positions
                                Rot1 = spatial.transform.Rotation.from_rotvec(nvec*60.0*(np.pi/180.0))
                                Rot2 = spatial.transform.Rotation.from_rotvec(-nvec*60.0*(np.pi/180.0))

                                coords[n+1] = xpar + hbond_length*Rot1.apply(part_axis)
                                coords[n+2] = xpar + hbond_length*Rot2.apply(part_axis)

                            # SP3 hybridization
                            if (Nhydros+Nheavys)==4:

                                # To define bonding plane, we need two heavy-atom
                                # partners for parent atom
                                part1 = HeavyParts[n][0]
                                part2 = HeavyParts[n][1]

                                # xpar, xp1, and xp2 define the bonding plane
                                xp1 = newstruc.coordinates[part1]
                                xp2 = newstruc.coordinates[part2]
                                noop = np.cross(xp1-xpar, xp2-xpar)
                                noop /= np.linalg.norm(nvec)

                                # nvec is orthogonal to the bonding plane of the two hydrogens
                                nvec = np.cross(part_axis, noop)
                                nvec /= np.linalg.norm(nvec)

                                Rot1 = spatial.transform.Rotation.from_rotvec(nvec*0.5*109.5*(np.pi/180.0))
                                Rot2 = spatial.transform.Rotation.from_rotvec(-nvec*0.5*109.5*(np.pi/180.0))

                                coords[n+1] = xpar + hbond_length*Rot1.apply(part_axis)
                                coords[n+2] = xpar + hbond_length*Rot2.apply(part_axis)


                        # Only SP3 hybridization is possible here
                        if Nhydros==3:

                            # part_axis points along the average axis of the 
                            # three hydrogen atoms. We need just to rotate one
                            # hydrogen atom by (180 - 109.5) degrees (along any 
                            # axis perpendicular to part_axis) and then rotate 
                            # the position of the other two by +/-120 degrees 
                            # around the part_axis (starting from the first H 
                            # position). 

                            # For concreteness, we'll define the initial 
                            # rotation axis using the cross product of 
                            # the parent atom, its first heavy-atom
                            # bond partner, and the partern's first partner.
                            part1 = HeavyParts[n][0]
                            part2 = HeavyParts[part1][0]
                            xp1 = newstruc.coordinates[part1]
                            xp2 = newstruc.coordinates[part2]
                            nvec = np.cross(xpar-xp1, xp2-xp1)
                            nvec /= np.linalg.norm(nvec)

                            # Rot1 rotates the first H-atom position to be 109.5 degrees from part_axis
                            Rot1 = spatial.transform.Rotation.from_rotvec((180.0-109.5)*(np.pi/180.0)*nvec)

                            # dX1 is a unit vector 109.5 degrees from part_axis
                            dX1 = Rot1.apply(part_axis.copy())

                            # Rot2 and Rot3 rotate dX1 by +/-120 degrees around part_axis
                            Rot2 = spatial.transform.Rotation.from_rotvec((120.0)*(np.pi/180.0)*part_axis)
                            Rot3 = spatial.transform.Rotation.from_rotvec((-120.0)*(np.pi/180.0)*part_axis)

                            coords[n+1] = xpar + hbond_length*dX1
                            coords[n+2] = xpar + hbond_length*Rot2.apply(dX1)
                            coords[n+3] = xpar + hbond_length*Rot3.apply(dX1)

                newstruc.coordinates = coords
                pigstruc = pigstruc + newstruc
                
                if MAKE_POSRE:
                    with open(prfile, 'w') as prfd:
                        prfd.write('[ position_restraints ]\n')
                        prfd.write('; atom  type      fx      fy      fz\n')
                        
                        # step through atoms
                        for at in newstruc:
                            
                            # If not hydrogen, add position restraint
                            if at.name[0]!='H':
                                prfd.write(str(at.idx+1).rjust(6) + '     1  1000  1000  1000\n')
                
                mess += "\n"
                
    if error==False:
        
        # Write the new structure to a *.pdb file
        pigstruc.write_pdb('pigmentsh.pdb')
        
        # And use gromacs to convert to *.gro
        out = !{"gmx editconf -f pigmentsh.pdb -o pigments.gro"}
    return error, mess

In [ ]:
# build_complex() produces combined gro and pdb files for the full complex
def build_complex():

    topin = 'topol.top'

    protfile = 'protein.gro'
    pgrofile = 'pigments.gro'

    cfile = 'complex.gro'
    topout = 'complex.top'
    cpdbfile = 'pdb2gmx.pdb'

    # Build top file
    pigstruc = pmd.load_file(pgrofile)
    PigList = []
    PigTypes = []
    for res in pigstruc.residues:
        PigList.append(res.name)
        if PigTypes.count(res.name)==0:
            PigTypes.append(res.name)

    with open(topin, 'r') as ifd:
        wrote_include = False
        with open(topout, 'w') as ofd:
            for line in ifd:
                ofd.write(line)
                if line[0:8]=='#include' and wrote_include==False:
                    for name in PigTypes:
                        ofd.write('#include \"'+name+'.itp\""\n')
                        ofd.write('#ifdef POSRES\n')
                        ofd.write('#include \"posre_'+name+'.itp\"\n')
                        ofd.write('#endif\n\n')
                        wrote_include = True

            for name in PigTypes:
                numstr = str(PigList.count(name))
                while len(numstr)+len(name)<21:
                    numstr = " " + numstr
                ofd.write(name + numstr)
            ofd.close()
        ifd.close()


    # Build gro file
    count = -1
    NPigAtoms = -1
    PigLines = []
    with open(pgrofile) as pfd:
        for line in pfd:

            # With count<NPigAtoms+1, we include box vector from pigment file.
            # Should be zeros since we just generated the gro file from a PDB. 
            # If NPigAtoms is already set and we aren't done yet.
            if NPigAtoms>0 and count<NPigAtoms+1:
                PigLines.append(line)
                count += 1

            if len(line.split())==1 and line.split()[0].isdigit():
                NPigAtoms = int(line.split()[0])
                count = 0
        
    count = -1
    NProAtoms = -1
    ProLines = []
    HeadLine = ''
    with open(protfile) as pfd:
        for line in pfd:

            # Record first line as header
            if len(HeadLine)==0:
                HeadLine = line

            # With count<NProAtoms, we do NOT include the box vector.
            # If NProAtoms is already set and we aren't done yet.
            if NProAtoms>0 and count<NProAtoms:
                ProLines.append(line)
                count += 1

            if len(line.split())==1 and line.split()[0].isdigit():
                NProAtoms = int(line.split()[0])
                count = 0
        
        
#     # Update pigment atom numbers to be sequential with protein.
#     # At the same time, write posre file for pigment.
#     atcount = NProAtoms+1
#     with open('pigres.itp', 'w') as rfd:
        
#         rfd.write('[ position_restraints ]\n')
#         rfd.write('; atom  type      fx      fy      fz\n')
        
#         # Loop through lines and update atom numbers one at a time
#         for L in range(0, len(PigLines)):
#             line = PigLines[L]
#             if len(line)>20:

#                 # If this looks like an atom entry
#                 if line[0:5].strip().isdigit() and line[15:20].strip().isdigit():

#                     # Right-justify to 5 chars
#                     numstr = str(atcount).rjust(5)

#                     # Update atom number to be sequential with protein atoms
#                     PigLines[L] = line[0:15] + numstr + line[20:]
                    
#                     # If it isn't a hydrogen, write a position restraint
#                     if line[10:15].strip()[0]!='H':
#                         rfd.write(str(atcount).rjust(6) + '     1  1000  1000  1000\n')

#                     # Update atcount
#                     atcount += 1

    #out = !{"cp " + protfile + " " + cfile}
    with open(cfile, 'w') as cfd:

        # Add header
        cfd.write(HeadLine)

        # Write total number of atoms
        cfd.write(str(NProAtoms+NPigAtoms)+"\n")

        # Add protein structure
        cfd.writelines(ProLines)

        # Add pigment structure
        cfd.writelines(PigLines)
        

In [ ]:
def minimize_hydrogens():
    
    topin = 'topol.top'

    protfile = 'protein.gro'
    pgrofile = 'pigments.gro'

    cfile = 'complex.gro'
    topout = 'complex.top'
    cpdbfile = 'pdb2gmx.pdb'
    
    print('Optimizing hydrogen atom positions. (Minimizing energy with heavy-atom restraints.)')
    
    # Generate gro and tpr files for minimizing H atom positions
    # The larger box in the gro file is needed to avoid "box too small" errors.
    out = runbash("gmx editconf -f " + cfile + " -o newbox.gro -bt cubic -d 1", 
                  "Padding cubic box with 1 nm empty space on all sides")
    out = runbash("gmx grompp -v -f hmin.mdp -c newbox.gro -r newbox.gro -p " + topout + " -o hmin.tpr",
                  "Compiling GROMACS binary input")
    out = runbash("gmx mdrun -v -deffnm hmin",
                  "Running energy minimization for hydrogen atoms (restrained heavy atoms)")
    
    # Generate charge file
    out = runbash("echo -e 0 | gmx trjconv -s hmin.tpr -f hmin.gro -o hmin.pdb",
                  "Generating PDB output for charge file.")
    out = runbash("gmx editconf -f hmin.tpr -mead -o charge.pdb",
                  "Using gmx editconf -mead to output pqr file including charges.")
    out = runbash("bash parse_pqr.sh mead.pqr",
                  "Parsing pqr file to extract charges")
    
    # Add any entries with no chain to chain 0.
    out = runbash("cp hmin.pdb " + cpdbfile, 
                  "Copying H-minimized output file to pdb2gmx.pdb for viewing")
    writelog(["Associating with chain 0 all residues that have not yet been assigned a chain.",
             "Final structure stored in pdb2gmx.pdb"], "# ")
    with open(cpdbfile, 'w') as ofd:
        with open('hmin.pdb') as ifd:
            for line in ifd:
                if len(line)>22:
                    if line[21]==' ':
                        line = line[:21] + '$' + line[22:]
                ofd.write(line)
                
    # Now we want to generate a potential energy curve for validation. 
    # But first we have to find out which index in gmx energy corresponds to the potential. 
    # Send a dummy command to gmx energy that will throw an error but let us see the 
    # options for output. 
    out = !{"echo -e 0 | gmx energy -f hmin.edr"}
    addtext = False
    SelText = ''
    # Parse through the output 
    for line in out:
        if line[0:10]=='----------':
            if addtext:
                break
            else:
                addtext = True
        else:
            if addtext:
                SelText += line + " "
    try: 
        # search the string for an occurance of "x Potential y" where x and y are nonnegative integers
        hit = re.match('.*[ \t]+([0-9]+[ \t]+)(Potential)[ \t]+[0-9]+.*', SelText)
        
        # The first group should be the number for the Potential Energy
        PotNum = hit.group(1).strip()
        
        # Now run the real potential energy command
        runbash("echo -e "+PotNum+" 0 | gmx energy -f hmin.edr -o potential.xvg", "Printing potential energy trajectory to file potential.xvg")
        
        # Extract title from plot.
        title = runbash("sed -n \'s/\(@ s0 legend \"\)\(.*\)\"/\\2/p\' potential.xvg", 'Extracting plot title')[0]
        yunits = runbash("sed -n \'s/\(@    yaxis  label \"\)\(.*\)\"/\\2/p\' potential.xvg", 'Extracting y-axis units')[0]
        xlabel = runbash("sed -n \'s/\(@    xaxis  label \"\)\(.*\)\"/\\2/p\' potential.xvg", 'Extracting x-axis units')[0]
        
        display_markdown('# ' + title + ' #', raw=True)
        display_markdown('Please check for consistency. You should see a smooth decay.', raw=True)
        fig = plt.figure(figsize=(6,4))

        dat = np.loadtxt('potential.xvg', comments=['@', '#'])
        
        plt.plot(dat[:,0], dat[:,1])
        plt.xlabel(xlabel, fontsize=16)
        plt.ylabel(title + ' ' + yunits, fontsize=16)
        plt.xticks(fontsize=16)
        plt.yticks(fontsize=16)
        plt.tight_layout()
        plt.show()
        
    # If for some reason parsing fails, print an error and skip potential energy output. 
    except:
        print('Error identifying Potential energy index in gmx energy. Cannot print potential energy curve.')
        writelog(['Error identifying Potential energy index in gmx energy. Cannot print potential energy curve.'])
    

In [ ]:
sim_running = False

def sym_saw(n):
    return ((n%100-50)*((n%100)>50) - (n%100-50)*((n%100)<=50))/50.0

def show_waitbar(waitbar):
    count = 0
    while True:
        waitbar.value = sym_saw(count)
        count += 1
        time.sleep(0.05)
        
        global sim_running 
        if sim_running==False:
            break

def start_waitbar():
    global sim_running
    sim_running = True
    
    bar = widgets.FloatProgress(value=0.0, min=0.0, max=1.0)
    display(bar)
    progress_thread = threading.Thread(target=show_waitbar, args=(bar,))
    progress_thread.start()
    
    return bar, progress_thread
    
def stop_waitbar(bar, thread):
    global sim_running
    sim_running = False
    thread.join()
    bar.close()
    del bar


writelog(["Adding hydrogens according to *.itp file bonding patterns.",
          "Any H atoms present in the input structure will be ignored"], "# ")
err, mess = add_hydrogens()
if err:
    print(mess)
else:
    writelog(["Hydrogens successfully added. Please check final structures for consistency.",
             "Now combining pigment and protein structure/topology files."], "# ")

    build_complex()

    struc = pmd.load_file('pigments.gro')
    display_markdown('Hydrogens have been added to each residue based on itp file bonding parameters.', raw=True)
    display_markdown('Any initial hydrogens present in the structure have been stripped.', raw=True)
    display_markdown('Please check structures displayed below for consistency.', raw=True)
    
    for res in struc.residues:
        if NGL_DEF:
            view = nv.NGLWidget()
            view.camera = 'orthographic'
            view._set_size('500px', '500px')
            view.add_trajectory(struc[':'+str(res.number)])
        else:
            view = widgets.HTML(value='<p style=\"text-align:center; font-size:20px\"><br><br>Install NGLView library<br>to view structures.</p>', 
                          layout=widgets.Layout(width='500px', height='500px'))
        display_markdown('## '+res.name + ' ' + str(res.number) + ' ##', raw=True)
        display(view)
        
    bar, thread = start_waitbar()
    minimize_hydrogens()
    stop_waitbar(bar, thread)

["sed: can't read potential.xvg: No such file or directory"]
